In [6]:
from pydantic_ai import Agent, RunContext, Tool
from langchain_community.retrievers import WikipediaRetriever

In [25]:
async def wiki_research(ctx: RunContext) -> str:
    wiki_retriever = WikipediaRetriever(top_k_results=1)
    """Search on wikipedia given a topic."""
    print(f"Dependencies: {ctx.deps}")
    wiki_retriever_results = await wiki_retriever.abatch([ctx.deps])
    wiki_docs = wiki_retriever_results
    return "\n\n".join(
        f"Title: {doc[0].metadata['title']}\n\nContent:\n {doc[0].page_content}\n\n"
        for doc in wiki_docs
    )


agent = Agent(
    "openai:gpt-4o-mini",
    deps_type=str,
    system_prompt=(
        "You are a research assistant that uses Wikipedia to find information.\n"
        "ALWAYS use the wiki_research tool with the provided deps to search Wikipedia and return the results.\n"
        "Do not ask clarifying questions - immediately use the wiki_research tool with whatever topic is provided in deps.\n"
    ),
    tools=[Tool(wiki_research, takes_ctx=True)],
)


In [27]:
res = await agent.run("Search Wikipedia for information", deps="climate change")
res.data

Dependencies: climate change


'I\'ve found information on climate change from Wikipedia. Here\'s a summary:\n\n**Climate Change Overview:**\n- Climate change refers to the ongoing increase in global average temperature, primarily driven by human activities like fossil fuel burning, deforestation, and certain agricultural and industrial practices. These activities release greenhouse gases, particularly carbon dioxide, which have increased in concentration by about 50% since the pre-industrial era.\n  \n**Environmental Impacts:**\n- The effects of climate change include expanding deserts, increased frequency of heat waves and wildfires, thawing permafrost, and retreating glaciers. It also leads to more intense storms, droughts, and other extreme weather events. Many species face relocation or extinction due to rapid environmental changes.\n\n**Human Health and Safety:**\n- Climate change poses significant threats to human health, increasing risks of flooding, extreme heat, food and water scarcity, disease, and econom

In [31]:
res.new_messages()

[ModelRequest(parts=[SystemPromptPart(content='You are a research assistant that uses Wikipedia to find information.\nALWAYS use the wiki_research tool with the provided deps to search Wikipedia and return the results.\nDo not ask clarifying questions - immediately use the wiki_research tool with whatever topic is provided in deps.\n', dynamic_ref=None, part_kind='system-prompt'), UserPromptPart(content='Search Wikipedia for information', timestamp=datetime.datetime(2025, 1, 20, 11, 22, 10, 703496, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'),
 ModelResponse(parts=[ToolCallPart(tool_name='wiki_research', args=ArgsJson(args_json='{}'), tool_call_id='call_wcKlpQP5OgDQxOYnghQpp8Gq', part_kind='tool-call')], timestamp=datetime.datetime(2025, 1, 20, 11, 22, 11, tzinfo=datetime.timezone.utc), kind='response'),
 ModelRequest(parts=[ToolReturnPart(tool_name='wiki_research', content='Title: Climate change\n\nContent:\n Present-day climate change includes both global